In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)

In [3]:
pyg_dataset = torch.load('../../data/curated dataset/cleaned_frag_pyg_dataset.pth')

C:\Users\sarab\AppData\Local\Temp\ipykernel_15888\3827785572.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load('../../data/curated dataset/cleaned

In [4]:
node_features = pyg_dataset[0].x.shape[1]  # Should be 134 based on your description
edge_features = pyg_dataset[0].edge_attr.shape[1]  # Should be 6 based on your description

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch_geometric.nn import SAGEConv, GINConv, GATConv

class VGAE(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(VGAE, self).__init__()
        self.encoder = VGEncoder(node_features, edge_features, hidden_dim, latent_dim)
        self.decoder = VGDecoder(latent_dim, hidden_dim, node_features, edge_features)

    def encode(self, x, edge_index, edge_attr):
        return self.encoder(x, edge_index, edge_attr)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * torch.clamp(logvar, -10, 10))
            eps = torch.randn_like(std)
            return mu + eps * std
        return mu

    def forward(self, x, edge_index, edge_attr):
        mu, logvar = self.encoder(x, edge_index, edge_attr)
        z = self.reparameterize(mu, logvar)
        reconstructed_x, reconstructed_edge_attr = self.decoder(z, edge_index)
        return reconstructed_x, reconstructed_edge_attr, mu, logvar

    def generate_multiple_outputs(self, x, edge_index, edge_attr, variator, num_samples=5):
        outputs = []
        with torch.no_grad():
            z, _ = self.encoder(x, edge_index, edge_attr)
            for _ in range(num_samples):
                mixed_noise = torch.randn_like(z) * variator
                z_noisy = z + mixed_noise
                outputs.append((*self.decoder(z_noisy, edge_index), edge_index))
        return outputs

class VGEncoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(VGEncoder, self).__init__()
        
        # First convolutional layer
        self.conv1 = GINConv(
            nn.Sequential(
                nn.Linear(node_features + edge_features, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.LeakyReLU(0.2)
            )
        )
        
        # Second convolutional layer
        self.conv2 = GINConv(
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.LeakyReLU(0.2)
            )
        )
        
        # Variational layers
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, edge_index, edge_attr):
        # Combine node and edge features
        edge_attr_expanded = torch.zeros((x.size(0), edge_attr.size(1))).to(x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        x = torch.cat([x, edge_attr_expanded], dim=1)
        
        # Convolutional layers
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        
        return self.fc_mu(x), self.fc_logvar(x)

class VGDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(VGDecoder, self).__init__()
        
        self.fc = nn.Linear(latent_dim, hidden_dim)
        
        self.node_decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, node_features)
        )
        
        self.edge_decoder = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, edge_features)
        )

    def forward(self, z, edge_index):
        h = F.leaky_relu(self.fc(z), 0.2)
        row, col = edge_index
        edge_h = torch.cat([h[row], h[col]], dim=1)
        return self.node_decoder(h), self.edge_decoder(edge_h)

In [6]:
def loss_function(recon_x, x, recon_edge_attr, edge_attr, mu, logvar):
    # Reconstruction losses
    node_recon_loss = F.mse_loss(recon_x, x)
    edge_recon_loss = F.mse_loss(recon_edge_attr, edge_attr)

    # KL divergence
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return node_recon_loss + edge_recon_loss + kl_divergence


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=60

In [8]:
import optuna
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader as PyGDataLoader

# Note: VGAE, VGEncoder, VGDecoder classes are already defined in the main code

def loss_function(recon_x, x, recon_edge_attr, edge_attr, mu, logvar):
    batch_size = x.size(0)
    
    # Reconstruction losses
    node_recon_loss = F.mse_loss(recon_x, x, reduction='sum')
    edge_recon_loss = 2.0 * F.mse_loss(recon_edge_attr, edge_attr, reduction='sum')
    
    # KL divergence 
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # Diversity enhancement
    repulsion_loss = -torch.pdist(mu).mean()
    
    # Total loss
    total_loss = (node_recon_loss + 
                 edge_recon_loss + 
                 0.1 * kl_divergence +
                 0.05 * repulsion_loss)
    
    return total_loss / batch_size

def objective(trial):
    # Define hyperparameters to search
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)

    # Prepare data loaders
    train_ratio = 0.8
    valid_ratio = 0.2

    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train

    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])
    train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = PyGDataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = VGAE(node_features, edge_features, hidden_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience // 2)

    best_val_loss = float('inf')
    epochs_without_improvement = 0

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            recon_x, recon_edge_attr, mu, logvar = model(data.x, data.edge_index, data.edge_attr)

            loss = loss_function(recon_x, data.x, recon_edge_attr, data.edge_attr, mu, logvar)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        # Validation step
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for val_data in val_loader:
                val_data = val_data.to(device)
                recon_x_val, recon_edge_attr_val, mu_val, logvar_val = model(
                    val_data.x, val_data.edge_index, val_data.edge_attr
                )
                val_loss = loss_function(
                    recon_x_val, val_data.x, recon_edge_attr_val, 
                    val_data.edge_attr, mu_val, logvar_val
                )
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        # Print training and validation losses for every epoch
        print(f"[Trial {trial.number}] Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"[Trial {trial.number}] Early stopping after {epoch + 1} epochs.")
            break

        scheduler.step(avg_val_loss)

    return best_val_loss

# Run the study with Optuna
if __name__ == "__main__":
    import multiprocessing

    num_jobs = multiprocessing.cpu_count()  # Use up to 7 cores
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=num_jobs)

    best_trial = study.best_trial
    print(f"\nBest Trial Number: {best_trial.number}")
    print("Best Parameters:", best_trial.params)
    print("Best Value (Objective):", best_trial.value)

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-01-06 01:19:12,930] A new study created in memory with name: no-name-97390be0-3d5d-447f-a8a0-03d201be3dd3


[Trial 7] Epoch 1/60, Training Loss: 33.1071, Validation Loss: 14.2785
[Trial 13] Epoch 1/60, Training Loss: 7.2208, Validation Loss: 2.8039
[Trial 15] Epoch 1/60, Training Loss: 13.0652, Validation Loss: 3.6644
[Trial 10] Epoch 1/60, Training Loss: 26.9631, Validation Loss: 9.8618
[Trial 3] Epoch 1/60, Training Loss: 9.3004, Validation Loss: 2.9619
[Trial 14] Epoch 1/60, Training Loss: 5.1470, Validation Loss: 1.8761
[Trial 1] Epoch 1/60, Training Loss: 12634.7619, Validation Loss: 13.1371
[Trial 11] Epoch 1/60, Training Loss: 17.3311, Validation Loss: 5.4767
[Trial 2] Epoch 1/60, Training Loss: 4321.6092, Validation Loss: 7.7899
[Trial 7] Epoch 2/60, Training Loss: 25.3690, Validation Loss: 12.4665
[Trial 13] Epoch 2/60, Training Loss: 2.4619, Validation Loss: 2.2166
[Trial 15] Epoch 2/60, Training Loss: 4.2538, Validation Loss: 2.6850
[Trial 0] Epoch 1/60, Training Loss: 6.8310, Validation Loss: 2.3344
[Trial 10] Epoch 2/60, Training Loss: 13.9464, Validation Loss: 7.1698
[Trial 7] 

[I 2025-01-06 01:34:48,459] Trial 1 finished with value: 3.3885840733846027 and parameters: {'hidden_dim': 320, 'latent_dim': 96, 'learning_rate': 0.03191582793146093, 'batch_size': 32, 'patience': 8}. Best is trial 1 with value: 3.3885840733846027.


[Trial 1] Epoch 21/60, Training Loss: 734.2060, Validation Loss: 6972.2233
[Trial 1] Early stopping after 21 epochs.
[Trial 14] Epoch 24/60, Training Loss: 0.6666, Validation Loss: 0.5901
[Trial 15] Epoch 32/60, Training Loss: 1.4089, Validation Loss: 1.0302
[Trial 2] Epoch 21/60, Training Loss: 1.6166, Validation Loss: 1.2377
[Trial 7] Epoch 41/60, Training Loss: 2.6198, Validation Loss: 1.9769
[Trial 11] Epoch 22/60, Training Loss: 1.7635, Validation Loss: 1.2682
[Trial 5] Epoch 7/60, Training Loss: nan, Validation Loss: nan
[Trial 16] Epoch 1/60, Training Loss: 1356360.7908, Validation Loss: 15.6516
[Trial 10] Epoch 31/60, Training Loss: 2.1026, Validation Loss: 2.1605
[Trial 13] Epoch 33/60, Training Loss: 0.6963, Validation Loss: 0.6178
[Trial 0] Epoch 16/60, Training Loss: 1.1537, Validation Loss: 0.8258
[Trial 7] Epoch 42/60, Training Loss: 2.5841, Validation Loss: 1.9594
[Trial 15] Epoch 33/60, Training Loss: 1.3971, Validation Loss: 1.0197
[Trial 3] Epoch 25/60, Training Loss:

[I 2025-01-06 01:38:15,462] Trial 8 finished with value: 14.401457917690276 and parameters: {'hidden_dim': 512, 'latent_dim': 96, 'learning_rate': 0.02570593674949771, 'batch_size': 8, 'patience': 4}. Best is trial 1 with value: 3.3885840733846027.


...

[Trial 197] Epoch 56/60, Training Loss: 0.5342, Validation Loss: 0.4895
[Trial 197] Early stopping after 56 epochs.
[Trial 198] Epoch 52/60, Training Loss: 0.6324, Validation Loss: 0.5311
[Trial 199] Epoch 49/60, Training Loss: 0.5474, Validation Loss: 0.4530
[Trial 198] Epoch 53/60, Training Loss: 0.6296, Validation Loss: 0.6114
[Trial 199] Epoch 50/60, Training Loss: 0.5530, Validation Loss: 0.4667
[Trial 198] Epoch 54/60, Training Loss: 0.6261, Validation Loss: 0.5561
[Trial 199] Epoch 51/60, Training Loss: 0.5527, Validation Loss: 0.4708
[Trial 198] Epoch 55/60, Training Loss: 0.6264, Validation Loss: 0.5393
[Trial 199] Epoch 52/60, Training Loss: 0.5503, Validation Loss: 0.4556
[Trial 198] Epoch 56/60, Training Loss: 0.6341, Validation Loss: 0.5487
[Trial 199] Epoch 53/60, Training Loss: 0.5496, Validation Loss: 0.4802
[Trial 198] Epoch 57/60, Training Loss: 0.6198, Validation Loss: 0.5203
[Trial 199] Epoch 54/60, Training Loss: 0.5458, Validation Loss: 0.4637
[Trial 198] Epoch 58

[I 2025-01-06 13:22:08,496] Trial 198 finished with value: 0.5202610691388448 and parameters: {'hidden_dim': 256, 'latent_dim': 32, 'learning_rate': 0.003539554833064167, 'batch_size': 32, 'patience': 9}. Best is trial 171 with value: 0.4382066955169042.


[Trial 198] Epoch 60/60, Training Loss: 0.6143, Validation Loss: 0.5590
[Trial 199] Epoch 57/60, Training Loss: 0.5345, Validation Loss: 0.4536


[I 2025-01-06 13:22:11,773] Trial 199 finished with value: 0.45304592450459796 and parameters: {'hidden_dim': 256, 'latent_dim': 32, 'learning_rate': 0.007825788237151509, 'batch_size': 32, 'patience': 9}. Best is trial 171 with value: 0.4382066955169042.


[Trial 199] Epoch 58/60, Training Loss: 0.5331, Validation Loss: 0.4628
[Trial 199] Early stopping after 58 epochs.

Best Trial Number: 171
Best Parameters: {'hidden_dim': 256, 'latent_dim': 32, 'learning_rate': 0.004250162901723459, 'batch_size': 32, 'patience': 9}
Best Value (Objective): 0.4382066955169042
